In [1]:
import numpy as np
import stim
from tqdm import tqdm
from cx_list_from_stabilizers_in_sequence import StabilizerCode
from interactive_cx_list_optimizer import InteractiveCxListOptimizer
import cProfile

from quits.qldpc_code import *
from quits.circuit import get_qldpc_mem_circuit
from quits.decoder import sliding_window_bposd_circuit_mem
from quits.simulation import get_stim_mem_result
from circuit_from_cx_list import memory_experiment_circuit_from_cx_list

In [2]:
lift_size, factor = 15, 3
p1 = [0, 1, 5]    # e + x + x^5
p2 = [0, 2, 7]    # e + x^2 + x^7

In [3]:
code = BpcCode(p1, p2, lift_size, factor)  # Define the BpcCode object
code.build_graph(seed=1)                   # Build the Tanner graph and assign directions to its edges. 

num_zcheck, num_data = code.hz.shape
num_xcheck, num_data = code.hx.shape
num_logical = code.lz.shape[0]
depth = sum(list(code.num_colors.values())) 
print('# data qubits: ', num_data, ' # logical qubits: ', num_logical)
print('# z-check qubits: ', num_zcheck, ' # x-check qubits: ', num_xcheck)
print('# layers of entangling gates: ', depth)

# data qubits:  90  # logical qubits:  8
# z-check qubits:  45  # x-check qubits:  45
# layers of entangling gates:  8


In [4]:
p = 2e-3           # physical error rate
num_rounds = 15    # number of rounds (T-1)
basis = 'Z'        # 'Z' or 'X'

circuit = stim.Circuit(get_qldpc_mem_circuit(code, p, p, p, p, num_rounds, basis=basis))

In [5]:
num_trials = 10
# Simulate the circuit using Stim. 
detection_events, observable_flips = get_stim_mem_result(circuit, num_trials, seed=1)   # simulate the circuit using Stim

W, F = 5, 3                     # sliding window parameters
max_iter, osd_order = 20, 10    # BP-OSD decoder parameters 

# Perform decoding of the detection_events generated from simulating the circuit. 
# Returns the logical observable flip predicted from decoding. 
logical_pred = sliding_window_bposd_circuit_mem(detection_events, circuit, code.hz, code.lz,\
                                                W, F, max_iter=max_iter, osd_order=osd_order, tqdm_on=True)

# Logical error is recorded whenever logical_pred does not match observable_flips for any logical qubit at any round
pL = np.sum((observable_flips- logical_pred).any(axis=1)) / num_trials
lfr = 1 - (1-pL)**(1/num_rounds)
print('p: %.7f, LFR: %.7f'%(p, lfr))

100%|██████████| 10/10 [00:01<00:00,  8.03it/s]

p: 0.0020000, LFR: 0.0000000


In [6]:
print(code.hx)
print(code.hz)

[[1 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 1]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 1]]
[[1 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [1 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 1]]


In [7]:
stabilizer_x = []
for i in range(code.hx.shape[0]):
    row = []
    for j in range(code.hx.shape[1]):
        if code.hx[i,j] == 1:
            row.append(j)
    stabilizer_x.append(row)
    
stabilizer_z = []
for i in range(code.hz.shape[0]):
    row = []
    for j in range(code.hz.shape[1]):
        if code.hz[i,j] == 1:
            row.append(j)
    stabilizer_z.append(row)

bpc_code = StabilizerCode(stabilizer_x, stabilizer_z, code.lx, code.lz)
cx_list = bpc_code.generate_cx_list()
ancilla_type, data_mapping, ancilla_mapping = bpc_code.build_mappings()
qubit_to_name = {v:k for k,v in data_mapping.items()}
qubit_to_name.update({v:k for k,v in ancilla_mapping.items()})

In [8]:
from permute_within_each_stabilizer import random_permutation_within_each_stabilizer
from permute_single_stabilizer import permute_single_stabilizer_inplace

In [9]:
random_permutation_within_each_stabilizer(cx_list)

In [10]:
def get_minimal_error():
    _, circ, idle_time = memory_experiment_circuit_from_cx_list(
                cx_list=cx_list,
                ancilla_type=ancilla_type,
                data_mapping=data_mapping,
                ancilla_mapping=ancilla_mapping,
                flag_mapping=dict(),  # No flag mapping used here
                logicals=code.lz,
                logical_type='Z',
                p_cx=0.,
                p_idle=0.,
                p_measurement_error=0.01,
                p_phenomenological_error=0.01,
                hook_errors=hook_errors,
                cycles_before_noise=1,
                cycles_with_noise=1,
                cycles_after_noise=1,
                flag=False
            )
    errors = circ.search_for_undetectable_logical_errors(
        dont_explore_edges_increasing_symptom_degree=False,
        dont_explore_detection_event_sets_with_size_above=7,
        dont_explore_edges_with_degree_above=9999,
        canonicalize_circuit_errors=True,
    )
    qubits_in_minimal_error = [error.circuit_error_locations[0].instruction_targets.targets_in_range[0].gate_target.value for error in errors]
    hook_ancillas_in_minimal_error = [qubit_to_name[qubit] for qubit in qubits_in_minimal_error if qubit in ancilla_mapping.values()]
    return hook_ancillas_in_minimal_error, len(errors)

In [12]:
# start with hook errors on all ancillas, then keep only hook errors on ancillas in minimal error
for _ in range(1000):
    hook_errors = {ancilla: [(1,0.01),(2,0.01),(3,0.01)] for ancilla, a_type in ancilla_type.items() if a_type == 'X'}
    bad_ancillas, distance = get_minimal_error()
    print(bad_ancillas)
    print('full distance: ', distance)
    hook_errors = {ancilla: [(1,0.01),(2,0.01),(3,0.01)] for ancilla in bad_ancillas}
    while True:
        a = np.random.choice(bad_ancillas)
        permute_single_stabilizer_inplace(cx_list, a)
        bad_ancillas, new_distance = get_minimal_error()
        print('new distance: ', new_distance)
        if new_distance > distance:
            break

KeyboardInterrupt: 

In [10]:
hook_errors = {ancilla: [(1,0.01),(2,0.01),(3,0.01)] for ancilla, a_type in ancilla_type.items() if a_type == 'X'}

In [17]:
largest_distance = 0
best_cx_list = cx_list.copy()
for _ in range(100):

    print(hook_ancillas_in_minimal_error)
    distance = len(errors)
    if distance < largest_distance:
        cx_list = best_cx_list.copy()
    elif distance > largest_distance:
        largest_distance = distance
        best_cx_list = cx_list.copy()
    else:
        for a in hook_ancillas_in_minimal_error:
            permute_single_stabilizer_inplace(cx_list, a)
    print('distance: ', distance)
print('largest distance: ', largest_distance)

Total idling time: 0
['X43', 'X27', 'X11']
distance:  6
Total idling time: 0
['X43', 'X27', 'X11']
distance:  6
Total idling time: 0
['X9', 'X14', 'X17', 'X27', 'X20']
distance:  6
Total idling time: 0
['X30', 'X32', 'X21', 'X23']
distance:  6
Total idling time: 0
['X17', 'X43', 'X35', 'X3', 'X30']
distance:  6
Total idling time: 0
['X20', 'X11', 'X9']
distance:  7
Total idling time: 0
['X20', 'X11', 'X9']
distance:  7
Total idling time: 0
['X15', 'X20', 'X36', 'X41']
distance:  7
Total idling time: 0
['X0', 'X40', 'X2', 'X11']
distance:  7
Total idling time: 0
['X42', 'X36', 'X7', 'X40']
distance:  6
Total idling time: 0
['X20', 'X11', 'X9']
distance:  7
Total idling time: 0
['X23', 'X32', 'X34', 'X43']
distance:  7
Total idling time: 0
['X20', 'X36', 'X41', 'X23']
distance:  6
Total idling time: 0
['X20', 'X11', 'X9']
distance:  7
Total idling time: 0
['X23', 'X32', 'X34', 'X43']
distance:  7
Total idling time: 0
['X15', 'X6', 'X43']
distance:  7
Total idling time: 0
['X15', 'X6']
di

KeyboardInterrupt: 

In [15]:
for ancilla, a_type in ancilla_type.items():
    if a_type == 'Z':
        continue
    hook_errors = {ancilla: [(1,0.01),(2,0.01),(3,0.01)]}
    largest_distance = 0
    best_cx_list = cx_list.copy()
    for _ in range(20):
        _, circ, idle_time = memory_experiment_circuit_from_cx_list(
                    cx_list=cx_list,
                    ancilla_type=ancilla_type,
                    data_mapping=data_mapping,
                    ancilla_mapping=ancilla_mapping,
                    flag_mapping=dict(),  # No flag mapping used here
                    logicals=code.lz,
                    logical_type='Z',
                    p_cx=0.,
                    p_idle=0.,
                    p_measurement_error=0.01,
                    p_phenomenological_error=0.01,
                    hook_errors=hook_errors,
                    cycles_before_noise=1,
                    cycles_with_noise=1,
                    cycles_after_noise=1,
                    flag=False
                )
        errors = circ.search_for_undetectable_logical_errors(
            dont_explore_edges_increasing_symptom_degree=False,
            dont_explore_detection_event_sets_with_size_above=7,
            dont_explore_edges_with_degree_above=9999,
            canonicalize_circuit_errors=True,
        )
        qubits_in_minimal_error = [error.circuit_error_locations[0].instruction_targets.targets_in_range[0].gate_target.value for error in errors]
        hook_ancillas_in_minimal_error = [qubit_to_name[qubit] for qubit in qubits_in_minimal_error if qubit in ancilla_mapping.values()]
        print(hook_ancillas_in_minimal_error)
        distance = len(errors)
        if distance < largest_distance:
            cx_list = best_cx_list.copy()
        elif distance > largest_distance:
            largest_distance = distance
            best_cx_list = cx_list.copy()
        else:
            for a in hook_ancillas_in_minimal_error:
                permute_single_stabilizer_inplace(cx_list, a)
        print('distance: ', distance)
    print('largest distance: ', largest_distance)

Total idling time: 0
['X0']
distance:  9
Total idling time: 0
['X0']
distance:  9
Total idling time: 0
['X0']
distance:  9
Total idling time: 0
['X0']
distance:  9
Total idling time: 0
['X0']
distance:  8
Total idling time: 0
['X0']
distance:  9
Total idling time: 0
['X0']
distance:  8
Total idling time: 0


KeyboardInterrupt: 